In [28]:
import os
import shutil
from pathlib import Path
from ipywidgets import interact

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


from core.Case import *
from core.globals import *
from core.Log import *


from methods.plots import *
from methods.preprocessing import *
from methods.intensity import *
from methods.slicing import *


CNTRL_dicom_root = "../Takotsubo-Syndrome/data/Inputs/normal_cases/"
TTS_dicom_root = "../Takotsubo-Syndrome/data/Inputs/takotsubo_cases/"
root_dir = "data/cases/"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
def loop_cases():
	#loop=0
	root_dir = Path("data/cases")
	for caseID in os.listdir(root_dir):
		if caseID.startswith('.'):
			continue
		case = Case(caseID)
		set_log(caseID)
		print(f"Processing {caseID}")
		ct = case.croppedCT
		if ct is None:
			continue
		bbox = bounding_box(case.totalsegs)
		print(f"Bounding box for {caseID}: {bbox}")
		path = os.path.join(case.casePath, "croppedCT.nii.gz")
		case.croppedCT = NiftiVolume.init_from_array(
			case.fullCT.data[bbox],
			case.fullCT.affine,
			case.fullCT.header,
			path)
		print(f"Cropped shape: {case.croppedCT.shape}")
		assignments = {
				"LV": case.LVtotalseg,
				"LA": case.LAtotalseg,
				"RV": case.RVtotalseg,
				"RA": case.RAtotalseg,
				"MYO": case.MYOtotalseg
			}
		for key, vol in assignments.items():
			cropped = NiftiVolume.init_from_array(
				vol.data[bbox],
				vol.affine,
				vol.header,
				os.path.join(case.casePath, "cropped", f"{key}.nii.gz")
			)
			setattr(case, f"{key}cropped", cropped)
			print(f"{key}cropped shape= {cropped.shape}")
		case.create_label_mask(case.croppedsegs, os.path.join(case.casePath, "label_mask.nii.gz"))





loop_cases()


Processing CNTRL_AAP_50415783_61F
Bounding box for CNTRL_AAP_50415783_61F: (slice(211, 381, None), slice(218, 341, None), slice(156, 186, None))
Cropped shape: (170, 123, 30)
LVcropped shape= (170, 123, 30)
LAcropped shape= (170, 123, 30)
RVcropped shape= (170, 123, 30)
RAcropped shape= (170, 123, 30)
MYOcropped shape= (170, 123, 30)
ct shape: (170, 123, 30) - segment LV shape: (170, 123, 30)
ct shape: (170, 123, 30) - segment LA shape: (170, 123, 30)
ct shape: (170, 123, 30) - segment RV shape: (170, 123, 30)
ct shape: (170, 123, 30) - segment RA shape: (170, 123, 30)
ct shape: (170, 123, 30) - segment MYO shape: (170, 123, 30)
Processing CNTRL_AB_10470318_90F
Bounding box for CNTRL_AB_10470318_90F: (slice(209, 390, None), slice(231, 369, None), slice(25, 102, None))
Cropped shape: (181, 138, 77)
LVcropped shape= (181, 138, 77)
LAcropped shape= (181, 138, 77)
RVcropped shape= (181, 138, 77)
RAcropped shape= (181, 138, 77)
MYOcropped shape= (181, 138, 77)
ct shape: (181, 138, 77) - seg

AttributeError: 'NoneType' object has no attribute 'data'

In [ ]:
root_dir = Path("data/cases")
CT_list = []
LV_list = []
MYO_list = []

loop=0
for caseID in os.listdir(root_dir):
	if caseID.startswith('.'):
		continue
	case = Case(caseID)
	set_log(caseID)
	casepath = root_dir / caseID
	infopath = casepath / "info.txt"
	#print(f"Processing {caseID}")
	## uncomment when info.txt content is fixed
	if infopath.exists():
		os.remove(infopath)
	try:
		croppedCT = case.croppedCT
		CT = intensity_stats(croppedCT)
		CT["caseID"] = caseID
		CT_list.append(CT)

		LVcropped = case.LVcropped
		LV = intensity_stats(LVcropped)
		LV["caseID"] = caseID
		LV_list.append(LV)

		MYOcropped = case.MYOcropped
		MYO = intensity_stats(MYOcropped)
		MYO["caseID"] = caseID
		MYO_list.append(MYO)
		#loop += 1
		#if loop > 5:
		#	break
	except Exception as e:
		print(f"Error processing: {e}")

save_intensity_stats(CT_list, filepath="data/cropped_intensities.csv")
save_intensity_stats(LV_list, filepath="data/LV_intensities.csv")
save_intensity_stats(MYO_list, filepath="data/MYO_intensities.csv")
#intensity_df = pd.DataFrame(stats_list)


CT_df = find_outliers(pd.DataFrame(CT_list)) #add column to df with is_outlier

new_outlier_df = detect_outliers(pd.DataFrame(CT_list)) #returns only outliers




#plot_intensity_distribution(intensity_df)


Error processing: 'NoneType' object does not support item assignment
Error processing: 'NoneType' object does not support item assignment
Error processing: 'NoneType' object does not support item assignment
Error processing: 'NoneType' object does not support item assignment
Error processing: 'NoneType' object does not support item assignment
Error processing: 'NoneType' object does not support item assignment
Error processing: 'NoneType' object does not support item assignment
Error processing: 'NoneType' object does not support item assignment
Error processing: 'NoneType' object does not support item assignment
Error processing: 'NoneType' object does not support item assignment


,min,max,mean,std,p0.5,p99.5,caseID
0,0.0,1.0,0.083958,0.277325,0.0,1.0,CNTRL_AAP_50415783_61F
1,0.0,1.0,0.082173,0.274627,0.0,1.0,CNTRL_AB_10470318_90F
2,0.0,1.0,0.090838,0.287379,0.0,1.0,CNTRL_AG_519880_37F
3,0.0,1.0,0.061930,0.241028,0.0,1.0,CNTRL_AIB_36444628_88F
4,0.0,1.0,0.068993,0.253443,0.0,1.0,CNTRL_AMB_08058471_74M
...,...,...,...,...,...,...,...
152,0.0,1.0,0.075607,0.264368,0.0,1.0,TTS_SKLL_5171467_81F
153,0.0,1.0,0.075863,0.264779,0.0,1.0,TTS_TS_2438216_76F
154,0.0,1.0,0.157636,0.364399,0.0,1.0,TTS_WF_10497121_62F
155,0.0,1.0,0.049811,0.217554,0.0,1.0,TTS_XZC_32444390_94F


In [ ]:
ctrlID = "CNTRL_BJB_42898_78F"
ttsID = "TTS_JET_40340663_84F"
case1 = Case(ctrlID)
case2 = Case(ttsID)
caseID1, ct_img1, mask_img1 = case1.caseID, case1.croppedCT.data, case1.LVcropped.data
caseID2, ct_img2, mask_img2 = case2.caseID, case2.croppedCT.data, case2.LVcropped.data
scrollable_ct_mask_compare("X", caseID1, ct_img1, mask_img1, caseID2, ct_img2, mask_img2)


interactive(children=(IntSlider(value=83, description='slice_index', max=166), Output()), _dom_classes=('widge…

In [ ]:
root_dir = Path("data/cases")
stats_list = []

loop=0
cases = os.listdir(root_dir)

spacings = []
for caseID in cases:
	if caseID.startswith('.'):
		continue
	#print(f"Processing {caseID}")
	case = Case(caseID)
	ct = case.croppedCT
	if ct is None:
		continue
	spacings.append(ct.spacing)  # tuple (sx, sy, sz)
spacings = np.array(spacings)
print("Mean spacing:", spacings.mean(axis=0))
print("Median spacing:", np.median(spacings, axis=0))
print("Range:", spacings.min(axis=0), spacings.max(axis=0))


Mean spacing: [0.71830285 0.7461681  1.7111081 ]
Median spacing: [0.703125 0.703125 1.25    ]
Range: [0.26171875 0.26171875 0.4       ] [0.976562  2.9999886 5.       ]


## Intensities: